In [8]:
%load_ext autoreload
%autoreload 2
import os
import copy
import pickle
from functools import lru_cache
import pandas as pd
import copy
import time
import sys
from collections import defaultdict
from statistics import mean

import numpy as np
import scipy
from scipy.spatial.distance import cosine as cosine_distance
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.append('../src')
from models import \
    get_expSG_1storder_relation_no_cache_NEW_ALLWORDS, \
    get_1storder_association_metric_fast, \
    get_matrices_from_term_lists, \
    calculate_association_metric_for_target_word, \
    word_set_to_mtx, save_pickle, open_pickle
from models import filter_terms_not_in_wemodel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

In [15]:
we_model = Word2Vec.load(we_model_dir+'/model.gensim')

print ('loading done!')

loading done!


In [17]:
FILEPATH = '../data/interim/association_metric_exps.pickle'

# Calculating *d*

In [18]:
# WEAT 3
EXP_NUM = 3
X_terms = ['roger','alan','frank','ian','justin',
          'ryan','andrew','fred','jack','matthew','stephen','brad','greg','jed',
          'paul','todd','brandon','hank','jonathan','peter','wilbur','amanda',
          'courtney','heather','melanie','sara','amber','crystal','katie',
          'meredith','shannon','betsy','donna','kristin','nancy','stephanie',
          'bobbie-sue','ellen','lauren','peggy','sue-ellen','colleen','emily',
          'megan','rachel','wendy']
Y_terms = ['alonzo','jamel','lerone','theo','alphonse','jerome','leroy',
           'torrance','darnell','lamar','lionel','rashaun','tyree','deion',
          'lamont','malik','terrence','tyrone','lavon','terryl',
          'wardell','aiesha','lashelle','nichelle','shereen','temeka','ebony',
          'latisha','shaniqua','tameisha','teretha','jasmine','laronya','shanise',
          'tanisha','tia','lakisha','latoya','sharise','tashika','yolanda',
          'lashandra','malika','shavonn','tawanda','yvette']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
          'loyal','pleasure','diamond','gentle','honest','lucky','rainbow',
          'diploma','gift','honor','miracle','sunrise','family','happy','laughter',
          'paradise','vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy','bomb',
          'divorce','jail','poverty','ugly','cancer','evil','kill','rotten','vomit']
X_terms, Y_terms = filter_terms_not_in_wemodel(we_model, X_terms, Y_terms)
A_terms, B_terms = filter_terms_not_in_wemodel(we_model, A_terms, B_terms)

The following terms were removed from the list first_list because they were not found in the we_model: ['bobbie-sue', 'sue-ellen']
The following terms were removed from the list second_list because they were not found in the we_model: ['terryl', 'aiesha', 'lashelle', 'temeka', 'tameisha', 'teretha', 'laronya', 'shanise', 'lakisha', 'sharise', 'tashika', 'lashandra', 'shavonn']
The following terms were removed from the first list to balance the length of the lists: ['roger', 'alan', 'frank', 'ian', 'justin', 'ryan', 'andrew', 'fred', 'jack', 'matthew', 'stephen']
The following terms were removed from the list first_list because they were not found in the we_model: []
The following terms were removed from the list second_list because they were not found in the we_model: []


# Generating List of Association Metrics

In [19]:
def get_all_relations(A_terms, B_terms, we_model):
    A_relations=get_expSG_1storder_relation_no_cache_NEW_ALLWORDS(A_terms, we_model)
    B_relations=get_expSG_1storder_relation_no_cache_NEW_ALLWORDS(B_terms, we_model)
    all_associations = np.mean(A_relations, axis=1) - np.mean(B_relations, axis=1)
    return all_associations

In [24]:
def get_1storder_association_metric_list_for_target_list(target_list, A_terms, B_terms, we_model):
    global ORDER
    ORDER = 'first'
    
    associations = np.array([])
    for word in tqdm(target_list):
        association = get_1storder_association_metric_fast(word, A_terms, B_terms, we_model)
        associations = np.append(associations, association)
    
    scaler = MinMaxScaler(feature_range=(-1,1))
    all_associations = get_all_relations(A_terms, B_terms, we_model)
    scaler.fit(all_associations.reshape(-1,1)) # Reshape is for a single feature, NOT for a single sample
    transformed = scaler.transform(associations.reshape(-1,1))
    return transformed.reshape(len(transformed))
metrics = get_1storder_association_metric_list_for_target_list(X_terms, A_terms, B_terms, we_model)
save_array(FILEPATH, metrics, EXP_NUM, ORDER)
print(metrics)

33


100%|████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 2113.42it/s]


[0.10917358 0.114474   0.03977669 0.21711674 0.02069786 0.14282087
 0.23171016 0.10893398 0.22164942 0.1731362  0.03183164 0.0843906
 0.13934326 0.11403528 0.10248176 0.15985223 0.36892195 0.13024954
 0.11066178 0.19458481 0.17395213 0.18826511 0.09246607 0.14566008
 0.14104248 0.12317581 0.17753852 0.17748386 0.11440853 0.06771086
 0.07495608 0.06568201 0.12313557]


# Second-Order

In [ ]:
def get_2ndorder_association_metric_list_for_target_list(target_list, A_terms, B_terms, we_model):
    global ORDER
    ORDER = 'second'
    
    [X_mtx, _, A_mtx, B_mtx] = get_matrices_from_term_lists(we_model, target_list, target_list, A_terms, B_terms)
    associations = np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, X_mtx)
    
    all_words_mtx = we_model.wv.vectors
    all_associations = np.array([])
    for x_vec in tqdm(all_words_mtx, desc='Getting association metric for all words'):
        metric = calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx)
        all_associations = np.append(all_associations, metric)
    
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler.fit(all_associations.reshape(-1,1)) # Reshape is for a single feature, NOT for a single sample
    transformed = scaler.transform(associations.reshape(-1,1))
    return transformed.reshape(len(transformed))
metric = get_2ndorder_association_metric_list_for_target_list(X_terms, A_terms, B_terms, we_model)
save_array(FILEPATH, metrics, EXP_NUM, ORDER)

Getting association metric for all words:  23%|██████▉                       | 71686/312425 [00:05<00:21, 11138.53it/s]

In [3]:
A = np.array([[1,1,1],[2,2,2]])
B = np.array([[3,3,3],[4,4,4]])
print(A)
print(B)
for (a,b) in zip(A,B):
    print(a)
    print(b)
    print('next')

[[1 1 1]
 [2 2 2]]
[[3 3 3]
 [4 4 4]]
[1 1 1]
[3 3 3]
next
[2 2 2]
[4 4 4]
next


In [22]:
np.linalg.norm(A, axis=0)

array([2.23606798, 2.23606798, 2.23606798])

In [5]:
A_dot_B = np.dot(A,B)

ValueError: shapes (2,3) and (2,3) not aligned: 3 (dim 1) != 2 (dim 0)

In [9]:
results_dict = defaultdict(dict)
save_pickle(results_dict, FILEPATH)

In [22]:
in_dict = open_pickle(FILEPATH)
in_dict

defaultdict(dict,
            {3: {'first': array([0.10917358, 0.114474  , 0.03977669, 0.21711674, 0.02069786,
                     0.14282087, 0.23171016, 0.10893398, 0.22164942, 0.1731362 ,
                     0.03183164, 0.0843906 , 0.13934326, 0.11403528, 0.10248176,
                     0.15985223, 0.36892195, 0.13024954, 0.11066178, 0.19458481,
                     0.17395213, 0.18826511, 0.09246607, 0.14566008, 0.14104248,
                     0.12317581, 0.17753852, 0.17748386, 0.11440853, 0.06771086,
                     0.07495608, 0.06568201, 0.12313557])}})

In [10]:
def save_array(FILEPATH, arr, exp_num, order):
    results_dict = open_pickle(FILEPATH)
    results_dict[exp_num][order] = arr
    save_pickle(results_dict, FILEPATH)